# 3.0 Combine data

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
from sklearn.preprocessing import Imputer
from sklearn.base import TransformerMixin
from sklearn import cross_validation
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.cross_validation import cross_val_score
import os

def sound():
    os.system('afplay /System/Library/Sounds/Sosumi.aiff')

%matplotlib inline
sns.set_context('notebook')

path = '/'.join(os.getcwd().split('/')[:-2])
print(path)

/Volumes/Dropbox/largetransfer/luc/carter


//anaconda/envs/data_sci/lib/python3.4/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
cols = ['Date','vacant','Survey Category','Survey Grade','SPA_NAME']

In [3]:
rng = pd.date_range('10/1/2009', periods=23, freq='3MS')

for date in rng:
    df0 = pd.read_csv(path+'/data/model_data/tci_2_0_'+str(date.year)+'_'+str(date.month)+'.csv', \
                      dtype={'Parcel ID':str}, index_col='parcel')
    df1 = pd.read_csv(path+'/data/model_data/tci_2_1_'+str(date.year)+'_'+str(date.month)+'.csv', \
                      dtype={'Parcel ID':str}, index_col='parcel')
    df2 = pd.read_csv(path+'/data/model_data/tci_2_2_'+str(date.year)+'_'+str(date.month)+'.csv', \
                      dtype={'Parcel ID':str}, index_col='parcel')
    df3 = pd.read_csv(path+'/data/model_data/tci_2_3_'+str(date.year)+'_'+str(date.month)+'.csv', \
                      dtype={'Parcel ID':str}, index_col='parcel')
    df = pd.merge(df0, df1[[col for col in df1.columns if col not in cols]], how='inner', left_index=True, right_index=True)
    df = pd.merge(df,  df2[[col for col in df2.columns if col not in cols]], how='inner', left_index=True, right_index=True)
    df = pd.merge(df,  df3[[col for col in df3.columns if col not in cols]], how='inner', left_index=True, right_index=True)
    
    df['price_per_sqft'] = df['tmktval']/df['totusabl']
    
    # df['missing_values'] = df.apply(lambda x: len(x[x.isnull()]),axis=1)
    to_remove = ['Parcel ID','parcel','Date','Survey Category','Survey Grade','date'\
             'condition','SPA_NAME','NAME10','style_filtered','rextwall','LENDER_PROCESS_TYPE']
    
    features = list(df.columns.values)
    for i in to_remove:
        if i in features:
            features.remove(i)

    break

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 114784 entries, 001-01-001 to 735-22-011
Data columns (total 46 columns):
vacant                    114784 non-null int64
Date                      114784 non-null object
Survey Category           114784 non-null object
Survey Grade              114784 non-null object
SPA_NAME                  114784 non-null object
cqual_num                 114784 non-null float64
occup                     114784 non-null float64
rnumstor                  114784 non-null float64
propsize                  114784 non-null float64
totusabl                  114784 non-null float64
tmktval                   114784 non-null float64
mktbldg                   114784 non-null float64
mktland                   114784 non-null float64
cond_val                  114784 non-null float64
style_filtered            114784 non-null object
ownerocc_value            114784 non-null float64
totbldgs                  114784 non-null float64
yrbuilt_filtered          114784 non-n

In [19]:
print(df[features].columns[df[features].dtypes == 'object'])

Index(['date'], dtype='object')


In [44]:
# transform categorical variables (including NaN's)
# objs = df[features].columns[df[features].dtypes == 'object']
# prefixes = ['style','ext','lender']

# objs = ['LENDER_PROCESS_TYPE']
# prefixes = ['lender']

# df2 = df[objs]
# df3 = pd.concat([pd.get_dummies(df2[col], prefix=prefixes[index]) for index, col in enumerate(df2)], axis=1)

# for i in objs:
#     if i in features:
#         features.remove(i)
df3 = pd.DataFrame()
df3[features] = df[features]
df3 = df3.fillna(0)

print(df3.shape)

(113132, 59)


In [48]:
df3.to_csv(path+'/data/model_data/tci_final.csv')